In [88]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdriv')

Drive already mounted at /content/gdriv; to attempt to forcibly remount, call drive.mount("/content/gdriv", force_remount=True).


In [89]:
import os
data_path = "/content/gdriv/My Drive/Colab Notebooks/data" 
data_name = os.listdir(data_path)

dt = pd.read_csv(data_path + '/fine_dust.csv')
pred_dt = pd.read_csv(data_path + '/fine_dust_without_y.csv')
print(dt.head())
print(pred_dt.head())

     PM   PM2      EG1       EG2       EG3  ...     EG20  HUM   TMP        WD   WS
0  27.0  25.0  1305.26  15069.82  12815.28  ...  1272.96   91   7.2  0.906308  0.6
1  52.0  17.0  1021.02  11788.14  10024.56  ...   954.72   36  14.3  0.438371  1.3
2  22.0  12.0   867.68  10017.76   8519.04  ...   636.48   38   6.7  0.992546  0.5
3  23.0  13.0   729.30   8420.10   7160.40  ...   318.24   23   9.7  0.241922  3.0
4  90.0  63.0   946.22  10924.54   9290.16  ...  1272.96   18   5.9  0.484810  1.3

[5 rows x 26 columns]
       EG1       EG2       EG3      EG4  ...  HUM   TMP        WD   WS
0   583.44   6736.08   5728.32  1272.96  ...   68  23.7  0.882948  0.7
1   875.16  10104.12   8592.48  1909.44  ...   34   3.5  0.190809  3.5
2   729.30   8420.10   7160.40  1591.20  ...   58  11.9  0.754710  0.6
3   875.16  10104.12   8592.48  1909.44  ...   25   7.0  0.275637  1.9
4  1237.94  14292.58  12154.32  2700.96  ...   66  13.7 -0.173648  0.4

[5 rows x 24 columns]


In [90]:
print(len(dt))
print(len(pred_dt))

3428
100


In [91]:
t_dt = pd.concat([dt, pred_dt])

In [92]:
# 미세먼지 등급 별로 구분
# PM2.5
idx0 = t_dt.loc[t_dt['PM2']<=15,:].index
idx1 = t_dt.loc[(t_dt['PM2']>=15) & (t_dt['PM2']<=35),:].index
idx2 = t_dt.loc[(t_dt['PM2']>=36),:].index


t_dt.iloc[idx0,1] = 0
t_dt.iloc[idx1,1] = 1
t_dt.iloc[idx2,1] = 2



t_dt['PM2'].value_counts()

1.0    1670
2.0    1033
0.0     725
Name: PM2, dtype: int64

In [93]:
t_dt

,PM,PM2,EG1,EG2,EG3,EG4,EG5,EG6,EG7,EG8,EG9,EG10,EG11,EG12,EG13,EG14,EG15,EG16,EG17,EG18,EG19,EG20,HUM,TMP,WD,WS
0,27.0,1.0,1305.26,15069.82,12815.28,2847.84,2847.84,1230.46,14206.22,12080.88,2684.64,2684.64,291.72,3368.04,2864.16,636.48,636.48,583.44,6736.08,5728.32,1272.96,1272.96,91,7.2,0.906308,0.6
1,52.0,1.0,1021.02,11788.14,10024.56,2227.68,2227.68,729.30,8420.10,7160.40,1591.20,1591.20,0.00,0.00,0.00,0.00,0.00,437.58,5052.06,4296.24,954.72,954.72,36,14.3,0.438371,1.3
2,22.0,0.0,867.68,10017.76,8519.04,1893.12,1893.12,1301.52,15026.64,12778.56,2839.68,2839.68,145.86,1684.02,1432.08,318.24,318.24,291.72,3368.04,2864.16,636.48,636.48,38,6.7,0.992546,0.5
3,23.0,0.0,729.30,8420.10,7160.40,1591.20,1591.20,729.30,8420.10,7160.40,1591.20,1591.20,0.00,0.00,0.00,0.00,0.00,145.86,1684.02,1432.08,318.24,318.24,23,9.7,0.241922,3.0
4,90.0,2.0,946.22,10924.54,9290.16,2064.48,2064.48,575.96,6649.72,5654.88,1256.64,1256.64,0.00,0.00,0.00,0.00,0.00,583.44,6736.08,5728.32,1272.96,1272.96,18,5.9,0.484810,1.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,NaN,1237.94,14292.58,12154.32,2700.96,2700.96,1806.42,20855.94,17735.76,3941.28,3941.28,291.72,3368.04,2864.16,636.48,636.48,1155.66,13342.62,11346.48,2521.44,2521.44,68,13.1,-0.997564,0.3
96,NaN,NaN,875.16,10104.12,8592.48,1909.44,1909.44,583.44,6736.08,5728.32,1272.96,1272.96,291.72,3368.04,2864.16,636.48,636.48,437.58,5052.06,4296.24,954.72,954.72,58,5.9,0.544639,2.0
97,NaN,NaN,1021.02,11788.14,10024.56,2227.68,2227.68,437.58,5052.06,4296.24,954.72,954.72,721.82,8333.74,7086.96,1574.88,1574.88,0.00,0.00,0.00,0.00,0.00,48,8.4,0.515038,1.3
98,NaN,NaN,583.44,6736.08,5728.32,1272.96,1272.96,729.30,8420.10,7160.40,1591.20,1591.20,1006.06,11615.42,9877.68,2195.04,2195.04,654.50,7556.50,6426.00,1428.00,1428.00,51,2.8,0.374607,0.6


In [94]:
# 데이터 셋 분류
x = t_dt.iloc[:,2:]
pm2 = t_dt.iloc[:,1]

print(x.head())
print(pm2.head())

       EG1       EG2       EG3      EG4  ...  HUM   TMP        WD   WS
0  1305.26  15069.82  12815.28  2847.84  ...   91   7.2  0.906308  0.6
1  1021.02  11788.14  10024.56  2227.68  ...   36  14.3  0.438371  1.3
2   867.68  10017.76   8519.04  1893.12  ...   38   6.7  0.992546  0.5
3   729.30   8420.10   7160.40  1591.20  ...   23   9.7  0.241922  3.0
4   946.22  10924.54   9290.16  2064.48  ...   18   5.9  0.484810  1.3

[5 rows x 24 columns]
0    1.0
1    1.0
2    0.0
3    0.0
4    2.0
Name: PM2, dtype: float64


In [95]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(x)
x = min_max_scaler.transform(x)

In [96]:
x = pd.DataFrame(x)
pm2 = pd.DataFrame(pm2)

In [97]:
ori_x = x.iloc[:len(dt),:]
ori_pm2 = pm2.iloc[:len(dt),:]

In [98]:
ori_x

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0.645102,0.645102,0.645102,0.645102,0.645102,0.629063,0.629063,0.629063,0.629063,0.629063,0.226087,0.226087,0.226087,0.226087,0.226087,0.445714,0.445714,0.445714,0.445714,0.445714,0.919192,0.446991,0.953150,0.109091
1,0.504621,0.504621,0.504621,0.504621,0.504621,0.372849,0.372849,0.372849,0.372849,0.372849,0.000000,0.000000,0.000000,0.000000,0.000000,0.334286,0.334286,0.334286,0.334286,0.334286,0.363636,0.650430,0.719164,0.236364
2,0.428835,0.428835,0.428835,0.428835,0.428835,0.665392,0.665392,0.665392,0.665392,0.665392,0.113043,0.113043,0.113043,0.113043,0.113043,0.222857,0.222857,0.222857,0.222857,0.222857,0.383838,0.432665,0.996273,0.090909
3,0.360444,0.360444,0.360444,0.360444,0.360444,0.372849,0.372849,0.372849,0.372849,0.372849,0.000000,0.000000,0.000000,0.000000,0.000000,0.111429,0.111429,0.111429,0.111429,0.111429,0.232323,0.518625,0.620932,0.545455
4,0.467652,0.467652,0.467652,0.467652,0.467652,0.294455,0.294455,0.294455,0.294455,0.294455,0.000000,0.000000,0.000000,0.000000,0.000000,0.445714,0.445714,0.445714,0.445714,0.445714,0.181818,0.409742,0.742385,0.236364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3423,0.752311,0.752311,0.752311,0.752311,0.752311,0.369025,0.369025,0.369025,0.369025,0.369025,0.675362,0.675362,0.675362,0.675362,0.675362,0.445714,0.445714,0.445714,0.445714,0.445714,0.383838,0.283668,0.320764,0.181818
3424,0.360444,0.360444,0.360444,0.360444,0.360444,0.298279,0.298279,0.298279,0.298279,0.298279,0.446377,0.446377,0.446377,0.446377,0.446377,0.222857,0.222857,0.222857,0.222857,0.222857,0.545455,0.518625,0.280760,0.163636
3425,0.500924,0.500924,0.500924,0.500924,0.500924,0.223709,0.223709,0.223709,0.223709,0.223709,0.000000,0.000000,0.000000,0.000000,0.000000,0.557143,0.557143,0.557143,0.557143,0.557143,0.555556,0.830946,0.320764,0.236364
3426,0.432532,0.432532,0.432532,0.432532,0.432532,0.298279,0.298279,0.298279,0.298279,0.298279,0.000000,0.000000,0.000000,0.000000,0.000000,0.665714,0.665714,0.665714,0.665714,0.665714,0.333333,0.083095,0.637791,0.454545


In [99]:
pred_x = x.iloc[len(dt):,:]
pred_pm2 = pm2.iloc[len(dt):,:]

In [100]:
pred_x

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
3428,0.288355,0.288355,0.288355,0.288355,0.288355,0.298279,0.298279,0.298279,0.298279,0.298279,0.339130,0.339130,0.339130,0.339130,0.339130,0.334286,0.334286,0.334286,0.334286,0.334286,0.686869,0.919771,0.941469,0.127273
3429,0.432532,0.432532,0.432532,0.432532,0.432532,0.588910,0.588910,0.588910,0.588910,0.588910,0.339130,0.339130,0.339130,0.339130,0.339130,0.445714,0.445714,0.445714,0.445714,0.445714,0.343434,0.340974,0.595374,0.636364
3430,0.360444,0.360444,0.360444,0.360444,0.360444,0.558317,0.558317,0.558317,0.558317,0.558317,0.000000,0.000000,0.000000,0.000000,0.000000,0.334286,0.334286,0.334286,0.334286,0.334286,0.585859,0.581662,0.877345,0.109091
3431,0.432532,0.432532,0.432532,0.432532,0.432532,0.818356,0.818356,0.818356,0.818356,0.818356,0.339130,0.339130,0.339130,0.339130,0.339130,0.222857,0.222857,0.222857,0.222857,0.222857,0.252525,0.441261,0.637791,0.345455
3432,0.611830,0.611830,0.611830,0.611830,0.611830,0.923518,0.923518,0.923518,0.923518,0.923518,0.226087,0.226087,0.226087,0.226087,0.226087,0.994286,0.994286,0.994286,0.994286,0.994286,0.666667,0.633238,0.413131,0.072727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3523,0.611830,0.611830,0.611830,0.611830,0.611830,0.923518,0.923518,0.923518,0.923518,0.923518,0.226087,0.226087,0.226087,0.226087,0.226087,0.882857,0.882857,0.882857,0.882857,0.882857,0.686869,0.616046,0.001142,0.054545
3524,0.432532,0.432532,0.432532,0.432532,0.432532,0.298279,0.298279,0.298279,0.298279,0.298279,0.226087,0.226087,0.226087,0.226087,0.226087,0.334286,0.334286,0.334286,0.334286,0.334286,0.585859,0.409742,0.772302,0.363636
3525,0.504621,0.504621,0.504621,0.504621,0.504621,0.223709,0.223709,0.223709,0.223709,0.223709,0.559420,0.559420,0.559420,0.559420,0.559420,0.000000,0.000000,0.000000,0.000000,0.000000,0.484848,0.481375,0.757501,0.236364
3526,0.288355,0.288355,0.288355,0.288355,0.288355,0.372849,0.372849,0.372849,0.372849,0.372849,0.779710,0.779710,0.779710,0.779710,0.779710,0.500000,0.500000,0.500000,0.500000,0.500000,0.515152,0.320917,0.687279,0.109091


In [101]:
ori_x = ori_x.to_numpy()
pred_x = pred_x.to_numpy()

In [102]:
pm2_trainx = ori_x
pm2_trainy = ori_pm2
pm2_pred_x = pred_x
pm2_pred_y = pred_pm2

In [103]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [104]:
# 네트워크 구조 정의

class mlp(nn.Module):

    def __init__(self, input_size):
        super(mlp, self).__init__()
        self.input_size = input_size
        self.fc1 = nn.Linear(self.input_size,30)
        self.bn1 = torch.nn.BatchNorm1d(30)
        self.fc2 = nn.Linear(30,9)
        self.bn2 = torch.nn.BatchNorm1d(9)
        self.fc4 = nn.Linear(9,3)
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout()
        self.sf = torch.nn.Softmax()
    
    def forward(self, x):
        output = self.fc1(x)
        output = self.bn1(output)
        output = self.relu(output)
        output = self.dropout(output)
        output = self.fc2(output)
        output = self.bn2(output)
        output = self.relu(output)
        output = self.dropout(output)
        output = self.fc4(output)
        output = self.sf(output)
        return output

In [105]:
# 입력 변수 torch tensor로 변경
pm2_trainx = torch.FloatTensor(pm2_trainx)
pm2_pred_x = torch.FloatTensor(pm2_pred_x)

In [107]:
# 출력 변수 torch tensor로 변경 (long 형태)
pm2_trainy = torch.FloatTensor(pm2_trainy.values).view(-1)
pm2_trainy = pm2_trainy.long()

pm2_pred_y = torch.FloatTensor(pm2_pred_y.values).view(-1)



In [108]:
# 네트워크 구조 호출
input_size = pm2_trainx.size()[1]
model = mlp(input_size)

In [109]:
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss = nn.CrossEntropyLoss()

In [112]:
for e in range(5000):
  optimizer.zero_grad()
  out = model(pm2_trainx)
  L = loss(out,pm2_trainy)
  L.backward()
  optimizer.step()



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [140]:
pred = model(pm2_pred_x)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [145]:
result = torch.max(pred,1)[1].numpy()

In [147]:
result = pd.DataFrame(result)

In [149]:
result_dt = pd.concat([pred_dt,result], axis=1)

In [152]:
result_dt.rename(columns = {0 : 'pred_pm2'}, inplace = True)

In [169]:
result_dt.to_csv("homework_9.csv")